# Imports

In [1]:
import librosa
import librosa.display
import os
import glob
import pickle

In [2]:
import itertools
import random
import numpy as np
import pandas as pd
import statistics 
import scipy.stats
import math

In [3]:
import tensorflow as tf
from tensorflow.keras import Input, layers, Model, utils, initializers, losses, optimizers, Sequential, callbacks, backend
from keras.utils import conv_utils
import tensorflow.keras as K

In [4]:
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score as recall
from sklearn.metrics import confusion_matrix as confusion

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd '/content/drive/My Drive/BTP - Dev Priya and Kushagra/Speech Emotion Recognition/'

/content/drive/My Drive/BTP - Dev Priya and Kushagra/Speech Emotion Recognition


In [7]:
%ls ./Dataset/

'=1.2.0'
 dataValid_1FTest_1M.pkl
 dataValid_1MTest_2F.pkl
 dataValid_2FTest_2M.pkl
 dataValid_2MTest_3F.pkl
 dataValid_3FTest_3M.pkl
 dataValid_3MTest_4F.pkl
 dataValid_4FTest_4M.pkl
 dataValid_4MTest_5F.pkl
 dataValid_5FTest_5M.pkl
 dataValid_5MTest_1F.pkl
 steam-train-whistle-daniel_simon-converted-from-mp3.wav


In [10]:
dataset_dir = 'Dataset'
ser_output_dir = 'Final Outputs/FishnetIEMOCAPd'

In [11]:
speaker_list = ['1F', '1M', '2F', '2M', '3F', '3M', '4F', '4M', '5F', '5M']
num_runs = 5
ser_acc = []
seed_value = None

# Model

## Custom Layer

In [12]:
class CustomAttention(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        self.hidden_size = kwargs.pop('hidden_size')
        super(CustomAttention, self).__init__(*args, **kwargs)
        self.W_omega = tf.Variable(
            initializers.GlorotNormal(seed=0)(shape=[self.hidden_size, 1]), 
            dtype=tf.float32,
            trainable=True,
            name="W_omega")
        self.b_omega = tf.Variable(
            initializers.GlorotNormal(seed=0)(shape=[1]), 
            dtype=tf.float32,
            trainable=True,
            name="b_omega")
        self.u_omega = tf.Variable(
            initializers.GlorotNormal(seed=0)(shape=[1]), 
            dtype=tf.float32,
            trainable=True,
            name="u_omega")
    
    def call(self, inputs, **kwargs):
        v = tf.sigmoid(tf.tensordot(inputs, self.W_omega, axes=1) + self.b_omega)
        vu = tf.tensordot(v, self.u_omega, axes=1)
        alphas = layers.Softmax()(vu)
        return tf.reduce_sum(inputs * tf.expand_dims(alphas, -1), 1)
        
    def get_config(self):
        config = ({
            'hidden_size': self.hidden_size 
        })
        base_config = super(CustomAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

## Build Model

### Fishnet 9.0
Trying to implement arch from paper itself

In [13]:
hparams = {
'OUT_CHANNELS' : [128],
'FILTER_CONV1' : 128,
'KERNEL_CONV1' : (3, 3),
'STRIDE_CONV1' : (1, 1),
'BIAS_INIT' : 'ones',
'KERNEL_INIT' : 'glorot_normal',
'PADDING_CONV' : 'SAME',
'BN_MOMENTUM' : 0.99,
'LEAKY_ALPHA' : 0.3,
'SIZE_POOL_CONV1' : (2, 4),
'PADDING_POOL_CONV1' : 'VALID',
'NUM_DIL_LAYERS' : 3,
'FILTER_CONV_UFLB' : 128,
'KERNEL_CONV_UFLB' :  (3, 3),
'STRIDE_CONV_UFLB' : (1, 1),
'DIL_RATE_CONV_UFLB' : (2, 2),
'PADDING_CONV_UFLB' : 'SAME',
'BILSTM_UNITS_SPEC' : 256,
'UNITS_FCN' : [512, 256, 64],
'LR' : 0.00001,
}

bottleneck function source: https://github.com/kevin-ssy/FishNet/blob/master/models/fish_block.py

In [14]:
def bottleneck(input, filters, mode="NORM", kernel_size=(3, 3), strides=(1, 1), dilation_rate=(1, 1)):
    inplanes = input.shape[-1]
    btnk_filters = filters // 4

    x = layers.BatchNormalization(momentum=hparams['BN_MOMENTUM'])(input)
    x = layers.Conv2D(filters=btnk_filters, kernel_size=(3, 3),
                      strides=(1, 1), dilation_rate=(1, 1), 
                      bias_initializer=hparams['BIAS_INIT'],
                      kernel_initializer=hparams['KERNEL_INIT'],
                      padding=hparams['PADDING_CONV'])(x)
    x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'])(x)

    x = layers.BatchNormalization(momentum=hparams['BN_MOMENTUM'])(x)
    x = layers.Conv2D(filters=btnk_filters, kernel_size=kernel_size,
                      strides=strides, dilation_rate=dilation_rate, 
                      bias_initializer=hparams['BIAS_INIT'],
                      kernel_initializer=hparams['KERNEL_INIT'],
                      padding=hparams['PADDING_CONV'])(x)
    x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'])(x)

    x = layers.BatchNormalization(momentum=hparams['BN_MOMENTUM'])(x)
    x = layers.Conv2D(filters=filters, kernel_size=(3, 3),
                      strides=(1, 1), dilation_rate=(1, 1), 
                      bias_initializer=hparams['BIAS_INIT'],
                      kernel_initializer=hparams['KERNEL_INIT'],
                      padding=hparams['PADDING_CONV'])(x)
    x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'])(x)

    if mode == "UP":
        y = None    #shortcut
    elif inplanes != filters or strides != (1, 1):
        y = layers.BatchNormalization(momentum=hparams['BN_MOMENTUM'])(input)
        y = layers.Conv2D(filters=filters, kernel_size=(3, 3),
                          strides=strides, dilation_rate=(1, 1), 
                          bias_initializer=hparams['BIAS_INIT'],
                          kernel_initializer=hparams['KERNEL_INIT'],
                          padding=hparams['PADDING_CONV'])(y)
        y = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'])(y)
    else:
        y = None

    return x, y


channelwise_reduction, ur_block, dr_block, se_block source: paper

In [15]:
# @tf.function
def channelwise_reduction(x, k):
    cin = x.shape[-1]
    cout = cin // k
    y = tf.reshape(x, [-1, x.shape[1], x.shape[2], cout, k])
    y = tf.reduce_sum(y, -1)
    return y

In [16]:
def ur_block(input, k=4, pool_size=(2, 2), kernel_size=(5, 3), strides=(1, 1), dilation_rate=(1, 1)):
    # input = layers.concatenate([input, residual])
    units = input.shape[-1]
    mx, _ = bottleneck(input, units // k, "UP", kernel_size, strides, dilation_rate)
    rx = channelwise_reduction(input, k)
    input = layers.Add()([mx, rx])
    input = layers.UpSampling2D(size=pool_size)(input)
    return input

In [17]:
def dr_block(input, k=1, pool_size=(2, 2), kernel_size=(5, 3), strides=(1, 1), dilation_rate=(1, 1)):
    # input = layers.concatenate([input, residual])
    units = input.shape[-1]
    mx, y = bottleneck(input, units // k, "NORM", kernel_size, strides, dilation_rate)
    if y != None:
        input = layers.Add()([mx, y])
    else:
        input = layers.Add()([mx, input])
    input = layers.MaxPool2D(pool_size=pool_size)(input)
    return input

In [18]:
def se_block(input, transform_fn=None, r=8):
    if transform_fn is not None:
        input = transform_fn(input)
    y = layers.GlobalAveragePooling2D()(input)
    units = y.shape[-1]
    y = layers.Dense(units=units // r, activation="linear")(y)
    y = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'])(y)
    y = layers.Dense(units=units, activation="sigmoid")(y) # Sigmoid activation
    input = tf.transpose(input, perm=[1,2,0,3])
    input = tf.math.multiply(input, y)
    input = tf.transpose(input, perm=[2,0,1,3])
    return input

In [19]:
input_shape = (300, 40, 3)

In [20]:
def conv_block(input, out_channels, kernel_size=(5, 3), strides=(1, 1)):
    x = layers.BatchNormalization(momentum=hparams['BN_MOMENTUM'])(input)
    x = layers.Conv2D(filters=out_channels, kernel_size=kernel_size, 
                      strides=strides,
                      bias_initializer=hparams['BIAS_INIT'],
                      kernel_initializer=hparams['KERNEL_INIT'],
                      padding=hparams['PADDING_CONV_UFLB'])(x)
    x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'])(x)
    return x

In [21]:
def build_model():
    model_input_spec = Input(shape=input_shape, name='spec_features')
    x = conv_block(model_input_spec, hparams['OUT_CHANNELS'][0])
    ############################# TAIL ########################################     
    x = dr_block(x, pool_size=(2, 1))
    t1 = x
    print(('t1', x.shape))

    x = dr_block(x, pool_size=(1, 2))
    t2 = x
    print(('t2', x.shape))

    x = dr_block(x, k=2, pool_size=(1, 2))
    t3 = x
    print(('t3', x.shape))

    x = dr_block(x, pool_size=(2, 1))
    t4 = x
    print(('t4', x.shape))
   
    x = se_block(x) # W x H x C3
    ############################# BODY ######################################## 
    
    x = ur_block(x, pool_size=(2, 1))
    x = layers.concatenate([t3, x]) # W x H x C2+C3
    b1 = x
    print(('b1', x.shape))

    x = ur_block(x, pool_size=(1, 2)) # 2W x 2H x (C2+C3)/k
    x = layers.concatenate([t2, x]) # 2W x 2H x C1+(C2+C3)/k
    b2 = x
    print(('b2', x.shape))

    x = ur_block(x, pool_size=(1, 2)) # 2W x 2H x (C2+C3)/k
    x = layers.concatenate([t1, x]) # 2W x 2H x C1+(C2+C3)/k
    b3 = x
    print(('b3', x.shape))

    ############################# HEAD ########################################
    x = dr_block(x, k=2, pool_size=(1, 2))
    x = layers.concatenate([b2, x]) 
    print(('h1', x.shape))
    
    x = dr_block(x, k=2, pool_size=(1, 2)) 
    x = layers.concatenate([b1, x]) 
    print(('h2', x.shape))

    x = dr_block(x, k=2, pool_size=(2, 1)) 
    x = layers.concatenate([t4, x]) 
    print(('h3', x.shape))

    #############################################################################

    time_step = x.shape[1]
    linear_units = x.shape[2]*x.shape[3]
    x = tf.reshape(x,[-1,time_step,linear_units])


    x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'],
                             name='LEAKY_LINEAR')(x)
    x = layers.Bidirectional(layers.GRU(units=hparams['BILSTM_UNITS_SPEC'],
                                         bias_initializer=hparams['BIAS_INIT'],
                                         kernel_initializer=hparams['KERNEL_INIT'], 
                                         return_sequences=True))(x)
    x = CustomAttention(hidden_size=x.shape[2])(x)

    for i, n in enumerate(hparams['UNITS_FCN']):
        x = layers.Dense(units=n, activation="linear", 
                         name='fcn_dense'+str(i+1))(x)
        x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'], 
                             name='fcn_leaky'+str(i+1))(x)
    
    x = layers.Dense(units=4, activation="softmax", name='Softmax')(x)

    model = Model(inputs=model_input_spec, outputs=x, name="model_adrnn") 

    model.compile(
        loss=losses.CategoricalCrossentropy(from_logits=False),
        optimizer=optimizers.Adam(
            learning_rate=hparams['LR']
        ),
        metrics=['categorical_accuracy'],
    )
    print(model.summary())

    return model

In [22]:
model = build_model()
# utils.plot_model(model, show_shapes=True)

('t1', TensorShape([None, 150, 40, 128]))
('t2', TensorShape([None, 150, 20, 128]))
('t3', TensorShape([None, 150, 10, 64]))
('t4', TensorShape([None, 75, 10, 64]))
('b1', TensorShape([None, 150, 10, 80]))
('b2', TensorShape([None, 150, 20, 148]))
('b3', TensorShape([None, 150, 40, 165]))
('h1', TensorShape([None, 150, 20, 230]))
('h2', TensorShape([None, 150, 10, 195]))
('h3', TensorShape([None, 75, 10, 161]))
Model: "model_adrnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 300, 40, 3)   12          spec_features[0][0]              
_____________________________________________________________________

# Retrieve Data

In [23]:
def retrieve_preprocessed_data():
    valid_actor = speaker_list[idx]
    test_actor = speaker_list[(idx+1)%10]
    file_code = 'Valid_' + valid_actor + 'Test_' + test_actor
    data_file_name = dataset_dir + '/data' + file_code + '.pkl'
    print(data_file_name)

    f = open(data_file_name, 'rb')
    output = pickle.load(f)
    train_features, train_labels = output[0], output[1]  
    test_features, test_labels, test_segments_per_utterance = output[2], output[3], output[5]
    valid_features, valid_labels, valid_segments_per_utterance = output[6], output[7], output[9]  
    f.close()

    train_features = tf.convert_to_tensor(train_features, dtype=tf.float32)
    valid_features = tf.convert_to_tensor(valid_features, dtype=tf.float32)
    test_features = tf.convert_to_tensor(test_features, dtype=tf.float32)

    train_labels = tf.one_hot(train_labels, 4, dtype=tf.float32)
    valid_labels = tf.one_hot(valid_labels, 4, dtype=tf.float32)
    test_labels = tf.one_hot(test_labels, 4, dtype=tf.float32)

    train_labels = tf.reshape(train_labels, [train_labels.shape[0], 4])
    valid_labels = tf.reshape(valid_labels, [valid_labels.shape[0], 4])
    test_labels = tf.reshape(test_labels, [test_labels.shape[0], 4])

    return train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance

In [24]:
def save_results(test_accuracy, test_conf):
    if not os.path.exists(ser_output_dir):
        os.mkdir(ser_output_dir)
    dir = os.path.join(ser_output_dir, 'Run_'+ str(run))
    if not os.path.exists(dir):
        os.mkdir(dir)
    test_actors = speaker_list[idx]
    valid_actors = speaker_list[idx-1][1:]
    file_code = 'Valid_' + ("_").join(valid_actors) + '_Test_' + ("_").join(test_actors) 
    data_file_name = dir + '/' + file_code + '.pkl'
    print(data_file_name)
    f = open(data_file_name, 'wb')
    pickle.dump((test_accuracy, test_conf), f)
    f.close()

# Train and Evaulate

In [25]:
def train_and_evaluate(model, num_epochs=1200):
    i=0
    best_valid_accuracy = 0
    best_epoch = 0

    while i< num_epochs:
        start = (i*60)%train_features.shape[0]
        end = min(start+60, train_features.shape[0])
        train_batch = train_features[start:end, :]
        train_batch_label = train_labels[start:end,:]
        loss = (model.train_on_batch(train_batch, train_batch_label, return_dict=True))

        if((i+1)%5==0):
            valid_acc_uw, valid_conf = evaluate(model, valid_features, 
                                                valid_segments_per_utterance, 
                                                valid_labels)
            
            if valid_acc_uw > best_valid_accuracy:
                best_epoch = i+1
                best_valid_accuracy = valid_acc_uw
                test_accuracy, test_conf = evaluate(model, test_features,
                                                    test_segments_per_utterance, 
                                                    test_labels)
                
                print('*'*30)
                print("Epoch: %05d" %(i+1))
                print(loss)
                print("Valid_UA: " + str(valid_acc_uw)) 
                print("Test UA: " + str(test_accuracy)) 

        if((i+1)%100 == 0):
            print('#'*15 + " Epoch: %05d " %(i+1) + '#'*15 )

        i += 1

    print('*'*30)
    print("Best Epoch: %05d" %(best_epoch))
    print("Best Valid Accuracy: " + str(best_valid_accuracy))
    print("Test_UA: " + str(test_accuracy))    
    print('Test Confusion Matrix:["ang","sad","hap","neu"]')
    print(test_conf)

    save_results(test_accuracy, test_conf)
    return 

In [26]:
def evaluate(model, spec_features, segments_per_utterance, labels, pooling='max'):
    y_pred = np.empty((len(segments_per_utterance),4),dtype=np.float32)
    y_pred_segments = model.predict(spec_features)
    index=0
    for j in range(len(segments_per_utterance)):
        if pooling == 'max':
            y_pred[j,:] = np.max(y_pred_segments[index:index+segments_per_utterance[j][0],:],0) 
        else:
            y_pred[j,:] = np.sum(y_pred_segments[index:index+segments_per_utterance[j][0],:],0)
        index+=(segments_per_utterance[j][0])

    acc_uw = recall(np.argmax(labels,1),np.argmax(y_pred,1),average='macro')
    conf = confusion(np.argmax(labels, 1),np.argmax(y_pred,1))
    return acc_uw, conf

# SER AVERAGE: Fishnet 9.0 

## Run 1 

In [27]:
run = 1

In [ ]:
idx = 0
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1FTest_1M.pkl
('t1', TensorShape([None, 150, 40, 128]))
('t2', TensorShape([None, 150, 20, 128]))
('t3', TensorShape([None, 150, 10, 64]))
('t4', TensorShape([None, 75, 10, 64]))
('b1', TensorShape([None, 150, 10, 80]))
('b2', TensorShape([None, 150, 20, 148]))
('b3', TensorShape([None, 150, 40, 165]))
('h1', TensorShape([None, 150, 20, 230]))
('h2', TensorShape([None, 150, 10, 195]))
('h3', TensorShape([None, 75, 10, 161]))
Model: "model_adrnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_35 (BatchNo (None, 300, 40, 3)   12          spec_features[0][0]              
_____________________________________

KeyboardInterrupt: ignored

In [ ]:
idx = 1
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1MTest_2F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_976 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 2
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2FTest_2M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1015 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 3
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2MTest_3F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1054 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 4
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_3FTest_3M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1093 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 5
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_3MTest_4F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1132 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 6
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_4FTest_4M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1171 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 7
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_4MTest_5F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1210 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 8
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_5FTest_5M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1249 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 9
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_5MTest_1F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1288 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
(0.7083333333333333 + 0.71875 + 0.7604166666666666 + )

## Run 2

In [ ]:
run = 2

In [ ]:
idx = 0
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1FTest_1M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_937 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 1
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1MTest_2F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_976 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 2
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2FTest_2M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1015 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 3
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2MTest_3F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1054 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 4
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_3FTest_3M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1093 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 5
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 6
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 7
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 8
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 9
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_5MTest_1F.pkl
('t1', TensorShape([None, 150, 40, 128]))
('t2', TensorShape([None, 150, 20, 128]))
('t3', TensorShape([None, 150, 10, 64]))
('t4', TensorShape([None, 75, 10, 64]))
('b1', TensorShape([None, 150, 10, 80]))
('b2', TensorShape([None, 150, 20, 148]))
('b3', TensorShape([None, 150, 40, 165]))
('h1', TensorShape([None, 150, 20, 230]))
('h2', TensorShape([None, 150, 10, 195]))
('h3', TensorShape([None, 75, 10, 161]))
Model: "model_adrnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_245 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
_____________________________________

In [ ]:
(0.7083333333333333 + 0.71875 + 0.7604166666666666 + )

## Run 3

In [ ]:
run = 3

In [ ]:
idx = 0
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1FTest_1M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_937 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 1
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1MTest_2F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_976 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 2
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2FTest_2M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1015 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 3
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2MTest_3F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1054 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 4
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_3FTest_3M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1093 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 5
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 6
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 7
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 8
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_5FTest_5M.pkl
('t1', TensorShape([None, 150, 40, 128]))
('t2', TensorShape([None, 150, 20, 128]))
('t3', TensorShape([None, 150, 10, 64]))
('t4', TensorShape([None, 75, 10, 64]))
('b1', TensorShape([None, 150, 10, 80]))
('b2', TensorShape([None, 150, 20, 148]))
('b3', TensorShape([None, 150, 40, 165]))
('h1', TensorShape([None, 150, 20, 230]))
('h2', TensorShape([None, 150, 10, 195]))
('h3', TensorShape([None, 75, 10, 161]))
Model: "model_adrnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_105 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
_____________________________________

In [ ]:
idx = 9
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_5MTest_1F.pkl
('t1', TensorShape([None, 150, 40, 128]))
('t2', TensorShape([None, 150, 20, 128]))
('t3', TensorShape([None, 150, 10, 64]))
('t4', TensorShape([None, 75, 10, 64]))
('b1', TensorShape([None, 150, 10, 80]))
('b2', TensorShape([None, 150, 20, 148]))
('b3', TensorShape([None, 150, 40, 165]))
('h1', TensorShape([None, 150, 20, 230]))
('h2', TensorShape([None, 150, 10, 195]))
('h3', TensorShape([None, 75, 10, 161]))
Model: "model_adrnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_210 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
_____________________________________

In [ ]:
(0.7083333333333333 + 0.71875 + 0.7604166666666666 + )

## Run 4

In [ ]:
run = 4

In [ ]:
idx = 0
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1FTest_1M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_937 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 1
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1MTest_2F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_976 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 2
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2FTest_2M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1015 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 3
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2MTest_3F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1054 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 4
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_3FTest_3M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1093 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 5
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 6
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 7
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 8
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_5FTest_5M.pkl
('t1', TensorShape([None, 150, 40, 128]))
('t2', TensorShape([None, 150, 20, 128]))
('t3', TensorShape([None, 150, 10, 64]))
('t4', TensorShape([None, 75, 10, 64]))
('b1', TensorShape([None, 150, 10, 80]))
('b2', TensorShape([None, 150, 20, 148]))
('b3', TensorShape([None, 150, 40, 165]))
('h1', TensorShape([None, 150, 20, 230]))
('h2', TensorShape([None, 150, 10, 195]))
('h3', TensorShape([None, 75, 10, 161]))
Model: "model_adrnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_175 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
_____________________________________

In [ ]:
idx = 9
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_5MTest_1F.pkl
('t1', TensorShape([None, 150, 40, 128]))
('t2', TensorShape([None, 150, 20, 128]))
('t3', TensorShape([None, 150, 10, 64]))
('t4', TensorShape([None, 75, 10, 64]))
('b1', TensorShape([None, 150, 10, 80]))
('b2', TensorShape([None, 150, 20, 148]))
('b3', TensorShape([None, 150, 40, 165]))
('h1', TensorShape([None, 150, 20, 230]))
('h2', TensorShape([None, 150, 10, 195]))
('h3', TensorShape([None, 75, 10, 161]))
Model: "model_adrnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_140 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
_____________________________________

In [ ]:
(0.7083333333333333 + 0.71875 + 0.7604166666666666 + )

## Run 5

In [ ]:
run = 5

In [ ]:
idx = 0
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1FTest_1M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_937 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 1
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_1MTest_2F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_976 (BatchN (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 2
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2FTest_2M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1015 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 3
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_2MTest_3F.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1054 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 4
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

Dataset/dataValid_3FTest_3M.pkl
('t1', TensorShape([None, 150, 40, 24]))
('t2', TensorShape([None, 150, 20, 24]))
('t3', TensorShape([None, 150, 10, 24]))
('t4', TensorShape([None, 75, 10, 48]))
('b1', TensorShape([None, 150, 10, 48]))
('b2', TensorShape([None, 150, 20, 48]))
('b3', TensorShape([None, 150, 40, 48]))
('h1', TensorShape([None, 150, 20, 96]))
('h2', TensorShape([None, 150, 10, 144]))
('h3', TensorShape([None, 75, 10, 192]))
Model: "Fishnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spec_features (InputLayer)      [(None, 300, 40, 3)] 0                                            
__________________________________________________________________________________________________
batch_normalization_1093 (Batch (None, 300, 40, 3)   12          spec_features[0][0]              
______________________________________________

In [ ]:
idx = 5
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 6
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 7
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 8
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
idx = 9
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance = retrieve_preprocessed_data()

model = build_model()
train_and_evaluate(model)

In [ ]:
(0.7083333333333333 + 0.71875 + 0.7604166666666666 + )

## Eval

In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
avg_accs = []
for run in [1, 2, 3, 4, 5]:    
    accs = []
    confs = []
    dir = os.path.join(ser_output_dir, 'Run_'+ str(run))
    if not os.path.exists(dir):
        continue
    for filename in os.listdir(dir):
        filename = os.path.join(dir, filename)
        f = open(filename, 'rb')
        test_accuracy, test_conf = pickle.load(f)
        accs.append(test_accuracy)
        confs.append(test_conf)
        f.close()
    avg_accs.append(np.mean(accs))
print(avg_accs)
m, l, r = mean_confidence_interval(avg_accs)
sdev = np.std(avg_accs)
print('SER AVERAGE: ' + str(m))
print('Standard Deviation: ' + str(sdev))
print('95% Confidence Interval: [' + str(l) + ', ' + str(r) + ']' )